In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

# 서울시 교통사고 시각화

In [12]:
traffic_seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [13]:
# 2019년 데이터만 추출하기
traffic_seoul_2019 = traffic_seoul[traffic_seoul.년도 == 2019]
traffic_seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [14]:
# index 정리하기
traffic_seoul_2019.reset_index(inplace=True, drop=True)
traffic_seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158


In [24]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기
traffic_analysis = traffic_seoul_2019.pivot_table(["발생건수", "부상자수", "사망자수"], 
                                                  aggfunc=sum, index="자치구명")
# pd.pivot_table(traffic_seoul_2019[["자치구명","발생건수", "부상자수", "사망자수"]], index="자치구명", aggfunc=sum)
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [30]:
# 위도, 경도 데이터 불러오기
seoul_limit = pd.read_csv("../Data/seoul.csv", encoding="euc-kr")
seoul_limit.columns = ["자치구명", "lon", "lat"]
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [34]:
# traffic_analysis와 seoul_limit 병합하기
data_result = pd.merge(left=traffic_analysis, right=seoul_limit, on="자치구명")
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [35]:
import folium

In [36]:
# folium은 int를 모르기 때문에 타입을 float로 바꿔줘야 한다.
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [39]:
data_result[["발생건수", "부상자수", "사망자수"]] = data_result[["발생건수", "부상자수", "사망자수"]].astype("float64")

In [40]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


### 자치구의 구청을 표시하기

In [42]:
map = folium.Map(
                  location=[37.5502, 126.982],
                  zoom_start=11
)
for name, lat, lng in zip(data_result.자치구명, data_result.lat, data_result.lon):
  folium.Marker([lat, lng], popup=name).add_to(map)

map

In [69]:
map = folium.Map(
                  location=[37.5502, 126.982],
                  zoom_start=11
)

# 자치구별 교통사고 발생건수를 표시하기
for name, lat, lng, rad in zip(data_result.자치구명, data_result.lat, data_result.lon, data_result.발생건수):
  folium.Marker([lat, lng], popup=name).add_to(map)
  folium.CircleMarker(
    [lat, lng],
    radius = rad / 100, # 원의 반지름
    # color = "skyblue",
    # fill = True,
    fill_color = "magenta",
    fill_opacity = (rad / data_result.발생건수.max()) * 0.7,
    popup=name
    ).add_to(map)
map

In [95]:
# 자치구별 교통사고 사망건수 표시하기
map = folium.Map(
                  location=[37.5502, 126.982],
                  zoom_start=11
)

# 자치구별 교통사고 발생건수를 표시하기
for name, lat, lng, rad in zip(data_result.자치구명, data_result.lat, data_result.lon, data_result.사망자수):
  folium.Marker([lat, lng], popup=name).add_to(map)
#   if ((rad == data_result.사망자수.max()) | (rad == data_result.사망자수.min())):
#     folium.CircleMarker(
#       [lat, lng],
#       radius = rad * 1.9, # 원의 반지름
#       color = "red",
#       # fill = True,
#       fill_color = "darkred",
#       fill_opacity = 0.2,
#       popup=name
#       ).add_to(map)
folium.CircleMarker(
      [data_result.loc[data_result.사망자수 == data_result.사망자수.max(),"lat"],
       data_result.loc[data_result.사망자수 == data_result.사망자수.max(),"lon"]],
      radius = data_result.사망자수.max() * 1.9, # 원의 반지름
      color = "red",
      # fill = True,
      fill_color = "darkred",
      fill_opacity = 0.2,
      popup=name
      ).add_to(map)
folium.CircleMarker(
      [data_result.loc[data_result.사망자수 == data_result.사망자수.min(),"lat"], 
      data_result.loc[data_result.사망자수 == data_result.사망자수.min(),"lon"]],
      radius = data_result.사망자수.min() * 1.9, # 원의 반지름
      color = "red",
      # fill = True,
      fill_color = "darkred",
      fill_opacity = 0.2,
      popup=name
      ).add_to(map)
map